In [1]:
# environment setting 
# https://huggingface.co/course/chapter0/1?fw=pt

!pip install transformers sentencepiece datasets nltk evaluate torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 5.2 MB/s 
     |████████████████████████████████| 1.3 MB 41.6 MB/s 
     |████████████████████████████████| 441 kB 20.2 MB/s 
     |████████████████████████████████| 72 kB 662 kB/s 
     |████████████████████████████████| 163 kB 50.7 MB/s 
     |████████████████████████████████| 7.6 MB 27.4 MB/s 
     |████████████████████████████████| 212 kB 34.1 MB/s 
     |████████████████████████████████| 95 kB 1.5 MB/s 
     |████████████████████████████████| 115 kB 53.3 MB/s 
     |████████████████████████████████| 127 kB 46.6 MB/s 
     |████████████████████████████████| 115 kB 46.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: dill
    Found existing installation: dill 0.3.6
    Uninstalling dill-0.3.6:
      S

In [2]:
######
import transformers
import torch
import numpy as np
import nltk
nltk.download('punkt')

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset
from evaluate import load


# preprocssing. this encodes command and actions 
def preprocess_function(examples):

    model_inputs = tokenizer(examples["commands"], max_length=tokenizer.model_max_length, truncation=True)
    model_inputs = tokenizer(examples["commands"], max_length=tokenizer.model_max_length, padding="longest", truncation=True, return_tensors='pt')
     

    # Setup the tokenizer for targets
    # with tokenizer.as_target_tokenizer():
    labels = tokenizer(examples["actions"], max_length=tokenizer.model_max_length, padding="longest", truncation=True, return_tensors='pt')

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


raw_datasets = load_dataset("scan", "simple")

# random initialize 
np.random.seed(123)
torch.random.manual_seed(123)

### smoke test
# raw_datasets['train'] = raw_datasets["train"].shuffle(seed=42).select(range(500))
# raw_datasets['test'] = raw_datasets["test"].shuffle(seed=42).select(range(500))

# model name, this can be changed 
model_checkpoint = "t5-small" 
model_name = model_checkpoint.split("/")[-1]

# define tokenizer, model 
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

# preprocess dataset
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)  

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Generating train split:   0%|          | 0/16728 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4182 [00:00<?, ? examples/s]

Dataset scan downloaded and prepared to /root/.cache/huggingface/datasets/scan/simple/1.0.0/68e9bb6106d45ba696314f2dea52a1e893f99cbf422feebbeda2f1d11878e020. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

...

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5_fast.py:166: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


Downloading:   0%|          | 0.00/242M [00:00<?, ?B/s]

  0%|          | 0/17 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

In [3]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['commands', 'actions', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 16728
    })
    test: Dataset({
        features: ['commands', 'actions', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 4182
    })
})

In [4]:
import transformers
import torch
import numpy as np
import nltk
nltk.download('punkt')

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset
from evaluate import load


# preprocssing. this encodes command and actions 
def preprocess_function(examples):

    model_inputs = tokenizer(examples["commands"], max_length=tokenizer.model_max_length, truncation=True)

    # Setup the tokenizer for targets
    # with tokenizer.as_target_tokenizer():
    labels = tokenizer(examples["actions"], max_length=tokenizer.model_max_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# training function, returns the model (which is trainer)
def model_train():
  # define args for finetuning
  batch_size = 8
  args = Seq2SeqTrainingArguments(
      f"{model_name}-scan-finetuned-yoon-1109",
      evaluation_strategy = "epoch",
      learning_rate=2e-5,
      per_device_train_batch_size=batch_size,
      per_device_eval_batch_size=batch_size,
      weight_decay=0.01,
      save_total_limit=3,
      num_train_epochs=20, # or 20
      predict_with_generate=True,
      # fp16=True,
      # push_to_hub=True,
  )

  # collator fixed of max length 
  data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

  trainer = Seq2SeqTrainer(
      model,
      args,
      train_dataset=tokenized_datasets["train"],
      eval_dataset=tokenized_datasets["test"],
      data_collator=data_collator,
      tokenizer=tokenizer,
  )

  trainer.train()

  return trainer    

# decoding function, this converts list of intergers into sentence 
def decode_data(prediction, label):
  decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  # Replace -100 in the labels as we can't decode them.
  labels = np.where(label_ids != -100, label_ids, tokenizer.pad_token_id)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  # We convert back into the sentence
  decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
  decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

  print('##################')
  print('decoded prediction: ', decoded_preds[0])
  print('decoded label: ', decoded_labels[0])

  return decoded_preds, decoded_labels

######### truncate function: THIS FUNCTION SHOULD BE REMOVED 
# this is actually wrong, as we are truncating the actual result to match the length
# We don't want to use this function, but this is needed as per now 
# def truncate_data(decoded_preds, decoded_labels):

#   test_preds = []
#   test_labels = []

#   for i in range(len(decoded_labels)):
#       min_length = min(len(decoded_preds[i]), len(decoded_labels[i]))
#       test_preds.append(decoded_preds[i][:min_length])
#       test_labels.append(decoded_labels[i][:min_length])

#   return test_preds, test_labels

# evaluate function: this calculates exact match score 
def evaluate_exact_match(decoded_preds, decoded_labels):
  exact_match_metric = load("exact_match")

  match = exact_match_metric.compute(predictions=decoded_preds, references=decoded_labels)['exact_match']
  print('Exact match: ', match)

  return match


if __name__ == "__main__":
  # dataset 
  raw_datasets = load_dataset("scan", "simple")

  # random initialize 
  np.random.seed(123)
  torch.random.manual_seed(123)
  
  ### smoke test
  # raw_datasets['train'] = raw_datasets["train"].shuffle(seed=42).select(range(500))
  # raw_datasets['test'] = raw_datasets["test"].shuffle(seed=42).select(range(500))
  
  # model name, this can be changed 
  model_checkpoint = "t5-small" 
  model_name = model_checkpoint.split("/")[-1]

  # define tokenizer, model 
  tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
  model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

  # preprocess dataset
  tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)  

  # train and predict 
  trainer = model_train()

  # creating max length among test labels: max_len = 253
  # max_length = 0
  # for i in tokenized_datasets['test']['labels']:
  #   if len(i) > max_length: 
  #     max_length = len(i)

  # predict: this needs to be fixed, but we are not sure how to work on it 
  # by default, it truncates the prediction into max_length = 20, but we are not sure why this is happening
  predictions, label_ids, _ = trainer.predict(tokenized_datasets["test"], max_length=tokenizer.model_max_length) # 512

  print('prediction: ', predictions[0])
  print('label: ', label_ids[0])
  

  # decoding results 
  decoded_preds, decoded_labels = decode_data(predictions, label_ids)  

  #### IMPORTANT #####################################
  #### truncate needed as per now, but want to figure out this part
  # test_preds, test_labels = truncate_data(decoded_preds, decoded_labels)
  # print('truncated_prediction: ', test_preds[0])
  # print('truncated_label: ', test_labels[0])

  # evaluate exact match result 
  match = evaluate_exact_match(decoded_preds, decoded_labels)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: actions, commands. If actions, commands are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 16728
  Num Epochs = 20
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 41820
  Number of trainable parameters = 60506624
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster 

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: ignored

In [ ]:
print('prediction: ', predictions[1])
print('label: ', label_ids[1])
print('length: ', len(predictions[1]), len(label_ids[1]))

prediction:  [    0    27   834 21164   567   834 27262    27   834 12054 22527    27
   834 21164   567   834 27262    27   834 12054 22527    27   834 21164
   567   834 27262    27   834 12054 22527    27   834 21164   567   834
 27262    27   834 12054 22527    27   834 21164   567   834 27262    27
   834 12054 22527    27   834 21164   567   834 27262    27   834 12054
 22527    27   834 21164   567   834 27262    27   834 12054 22527    27
   834 21164   567   834 27262    27   834 12054 22527    27   834 21164
   567   834 27262    27   834 12054 22527    27   834 21164   567   834
 27262    27   834 12054 22527    27   834 21164   567   834 27262    27
   834 12054 22527    27   834 21164   567   834 27262    27   834   448
  7443    27   834 21164   567   834 27262    27   834   448  7443    27
   834 21164   567   834 27262    27   834   448  7443    27   834 21164
   567   834 27262    27   834   448  7443    27   834 21164   567   834
 27262    27   834   448  7443    27  

In [ ]:
print('prediction: ', decoded_preds[0])
print('label: ', decoded_labels[0])
print('length: ', len(decoded_preds[0]), len(decoded_labels[0]))

prediction:  I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_TURN_LEFT I_TURN_LEFT
label:  I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_TURN_LEFT I_TURN_LEFT
length:  114 101


In [ ]:
print('prediction: ', decoded_preds[1])
print('label: ', decoded_labels[1])
print('length: ', len(decoded_preds[1]), len(decoded_labels[1]))

prediction:  I_TURN_RIGHT I_WALK I_TURN_RIGHT I_WALK I_TURN_RIGHT I_RUN I_TURN_RIGHT I_RUN
label:  I_TURN_RIGHT I_WALK I_TURN_RIGHT I_WALK I_TURN_RIGHT I_RUN I_TURN_RIGHT I_RUN
length:  77 77


In [ ]:
for i in range(len(decoded_labels)):
  if len(decoded_preds[i]) != len(decoded_labels[i]):
    print(i, len(decoded_preds[i]), len(decoded_labels[i]))

0 114 101
2 215 195
3 341 284
6 246 227
12 282 244
13 316 296
17 229 189
19 174 150
21 181 168
25 239 199
27 184 158
28 205 185
29 107 95
32 135 122
34 127 108
35 106 87
39 198 178
40 128 116
42 272 252
45 122 110
47 137 124
48 212 176
50 341 284
51 222 184
57 199 179
59 321 301
61 227 209
66 108 89
67 181 169
70 126 114
83 363 332
87 470 391
88 217 205
89 140 116
100 201 182
102 371 299
106 115 97
109 195 169
112 245 225
113 148 129
116 225 187
119 311 299
121 562 467
124 108 90
126 587 467
128 347 311
129 84 97
130 147 134
134 303 284
138 164 152
143 177 157
146 339 319
148 142 129
149 145 133
158 459 399
160 266 246
161 182 163
162 495 375
169 235 223
170 200 181
175 294 203
176 124 137
178 296 263
182 299 281
184 379 303
189 284 246
191 34 41
192 205 185
193 207 188
195 235 223
197 218 199
199 255 235
207 59 47
213 210 190
222 282 263
223 259 239
224 182 163
230 499 299
232 65 53
234 250 231
237 117 97
243 409 305
244 366 291
245 200 180
246 199 163
247 245 225
249 251 231
250 259 

In [ ]:
import gc
gc.collect()

66